<a href="https://colab.research.google.com/github/Sally-Ama-Sampson/Estimating-Tair-from-CS-generated-Data-/blob/main/morphometrics_Computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-------------------------------------------------------------------------------
# Program Name: Building morphological analysis
# Purpose: Morphometrics computed on top of building footprint polygons for morphological typology analysis

# This Python 3 environment mainly depends on Momepy, GeoPandas, PyGEOS, Scipy

# Version:     0.1
#              Functionalities:
#              1. Pre-processing building footprints;
#              2. Building morphometrics;
#              ?. Morphological clustering will be implemented once multiple cities are processed.

# Author:      Jiong (Jon) Wang, Angela Abascal, Stefanos Georganos, Martin Fleischmann
#
# Created:     02/05/2024
# Copyright:   (c) J. Wang, A. Abascal, S. Georganos, M. Fleischmann, 2024
# Licence:     MIT
#-------------------------------------------------------------------------------

# Urban Morphology With Building Morphometrics

There are two parts: (1) Preparation of urban elements for morphological analyis, and (2) Computation of morphometrics upon the prepared urban elements.

The input data for now is completely from the Google AI Open Buildings (@ https://sites.research.google/open-buildings/), but other GIS vector data about basic urban elements, e.g. streets and districts, can be used as well. For now, the building data from Google is considered only with its 2D form, but the height as the 3rd dimension can be involved in the future.


# Part I: Preparing Urban Elements For Morphometrics

In [ ]:
# Import necessary packages/modules. Most of the packages/modules used are imported here, and only a few others are imported along the codes below.

import math
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import momepy as mm
import osmnx as ox
import numpy as np
import libpysal
import scipy as sp
import mapclassify
import mapclassify.classifiers as classifiers
import contextily as cx
import utils

from shapely.geometry import box
from shapely import wkt
from tqdm import tqdm
from momepy import limit_range
from inequality.theil import Theil


### Load Raw Data from Google Open Buildings

In [ ]:
# Create Geopandas data frame from the *.csv downloaded from Google.
# Assign proper coordinate system.

google_building_file = 'harare.geojson'
city_name = 'Harare'

buildings = gpd.read_file(google_building_file)

'''
df = pd.read_csv(google_building_file)
df['geometry'] = df['geometry'].apply(wkt.loads)
buildings = gpd.GeoDataFrame(df, crs='epsg:4326')
'''


In [ ]:
# Check data formality by showing the first few lines of the dataframe table

buildings.head()


In [ ]:
# Check metadata to see if coordinates and projections are correct.

buildings.crs


In [ ]:
# Subset of columns as some columns are not necessary to keep for further processing.

buildings = buildings[['geometry']]
buildings.tail()


In [ ]:
# Load clip extent, to be defined.

'''
clip_extent = gpd.read_file('clip.shp').to_crs(4326)
print("clip extent coordinate:", clip_extent.crs)
'''
# All bounds are from the GHS Urban Centre database
global_bound = gpd.read_file("ucdb2019.gpkg")

# Extract a city boundary from the dataframe
bound = global_bound[global_bound.UC_NM_MN == city_name]

# add centroid
bound['centroid'] = bound['geometry'].centroid

# Overview extent
print(bound.crs)
ax = bound.plot(figsize=(10, 5), color='lightgrey', alpha=0.5, edgecolor='red', linewidth=1.0, facecolor=None)
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron) #crs='EPSG:4326',
ax.set_axis_off()

# Show more detail of the extent
bound


In [ ]:
# Clip buildings into extent

# buildings = gpd.clip(buildings, bound)

# Check the number of buildings within the extent
print(buildings.shape[0])


In [ ]:
# Check again by showing the clipped dataframe formality.

buildings.head()


In [ ]:
# Check again the clipped building metadata

buildings.crs


### Preprocess buildings

Mainly the Geometric correction

In [ ]:
# Delete incomplete polygons

buildings.geometry = buildings.buffer(0)
buildings = buildings[~buildings.geometry.isna()]
buildings = buildings.reset_index(drop=True).explode().reset_index(drop=True)


In [ ]:
# Quick statistics about the total

buildings.geom_type.value_counts()


In [ ]:
# Close holes

buildings = gpd.GeoDataFrame(geometry=utils.fill_insides(buildings))
buildings["uID"] = range(len(buildings))


In [ ]:
# Reproject building data into local UTM epsg

def convert_wgs_to_utm(lon: float, lat: float):
    """Based on lat and lng, return best utm epsg-code"""
    utm_band = str((math.floor((lon + 180) / 6 ) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0'+utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
        return epsg_code
    epsg_code = '327' + utm_band
    return epsg_code

local_epsg = convert_wgs_to_utm(bound['centroid'].x.values[0], bound['centroid'].y.values[0])


buildings = buildings.to_crs(local_epsg)
buildings.crs


In [ ]:
# Save the preprocessed building data

buildings.to_parquet('buildings.pq')


### Generate tessellation

In [ ]:
# Check building elements invalid for creating tessellation

check = mm.CheckTessellationInput(buildings)


In [ ]:
# Delete elements invalid for tessellation

buildings = buildings.drop(check.collapse.index.union(check.overlap.index).union(check.split.index))


In [ ]:
# Operate tessellation

import warnings
warnings.simplefilter('ignore')

limit = mm.buffered_limit(buildings, 100)
%time tess = mm.Tessellation(buildings, "uID", limit, segment=2).tessellation


In [ ]:
# Save tessellation results

tess.to_parquet('tessellation.pq')


In [ ]:
# Quick check tessellation total

tess.shape


In [ ]:
# Visualize

buildings.plot(figsize=(15,15))
tess.plot(figsize=(15,15))

In [ ]:
# Simple check if all buildings are considered in tessellation

if len(buildings) == len(tess):
    print('The buildings along with the tessellation are prepared!')
else:
    print('!ATTENTION!')


# Part II: Computing Morphometrics

Including *primary* and *contextual* level.

### Primary

Primary morphometrics are computed at locations of each building and tessellation element.

This is independent from Part I, but please do import necessary packages/modules by running the first block in Part I.

In [ ]:
# Read saved building and tessellation

blg = gpd.read_parquet('buildings.pq')
tess = gpd.read_parquet('tessellation.pq')


In [ ]:
# Simple building morphometrics

blg['sdbAre'] = mm.Area(blg).series
blg['sdbPer'] = mm.Perimeter(blg).series
blg['ssbCCo'] = mm.CircularCompactness(blg, 'sdbAre').series
blg['ssbCor'] = mm.Corners(blg).series
blg['ssbSqu'] = mm.Squareness(blg).series
blg['ssbERI'] = mm.EquivalentRectangularIndex(blg, 'sdbAre', 'sdbPer').series
blg['ssbElo'] = mm.Elongation(blg).series

cencon = mm.CentroidCorners(blg)
blg['ssbCCM'] = cencon.mean
blg['ssbCCD'] = cencon.std

blg['stbOri'] = mm.Orientation(blg).series
tess['stcOri'] = mm.Orientation(tess).series
blg['stbCeA'] = mm.CellAlignment(blg, tess, 'stbOri', 'stcOri', 'uID', 'uID').series


In [ ]:
# Simple tessellation morphometrics

tess['sdcLAL'] = mm.LongestAxisLength(tess).series
tess['sdcAre'] = mm.Area(tess).series
tess['sscCCo'] = mm.CircularCompactness(tess, 'sdcAre').series
tess['sscERI'] = mm.EquivalentRectangularIndex(tess, 'sdcAre').series
tess['sicCAR'] = mm.AreaRatio(tess, blg, 'sdcAre', 'sdbAre', 'uID').series


In [ ]:
# Morphometrics considering spatial neighborhood characteristics

queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tess, ids="uID", silence_warnings=True)

blg["mtbAli"] = mm.Alignment(blg, queen_1, "uID", "stbOri").series
blg["mtbNDi"] = mm.NeighborDistance(blg, queen_1, "uID").series
tess["mtcWNe"] = mm.Neighbors(tess, queen_1, "uID", weighted=True).series
tess["mdcAre"] = mm.CoveredArea(tess, queen_1, "uID").series


In [ ]:
# Save intermediate results

tess.drop(columns='geometry').to_parquet('tess_data.parquet')
blg.drop(columns='geometry').to_parquet('blg_data.parquet')


In [ ]:
# Morphometrics considering wider spatial neighborhood characters

queen_3 = mm.sw_high(k=3, weights=queen_1)

blg_q1 = libpysal.weights.contiguity.Queen.from_dataframe(blg, silence_warnings=True)

blg['ltbIBD'] = mm.MeanInterbuildingDistance(blg, queen_1, 'uID', queen_3, verbose=False).series
blg['ltcBuA'] = mm.BuildingAdjacency(blg, queen_3, 'uID', blg_q1, verbose=False).series


In [ ]:
# Update unique ID

tess = tess.merge(blg[['uID']], on='uID', how='left')


In [ ]:
# Save again some intermediate results

tess.drop(columns='geometry').to_parquet('tess_data.parquet')
blg.drop(columns='geometry').to_parquet('blg_data.parquet')

fo = libpysal.io.open('queen1.gal', 'w')
fo.write(queen_1)
fo.close()

fo = libpysal.io.open('queen3.gal', 'w')
fo.write(queen_3)
fo.close()

fo = libpysal.io.open('blg_queen.gal', 'w')
fo.write(blg_q1)
fo.close()


In [ ]:
# Join results

merged = tess.merge(blg.drop(columns=['geometry']), on='uID')


In [ ]:
# Save results for primary morphometrics

primary = merged.drop(columns=['geometry'])
primary.to_parquet('primary.parquet')


## Contextual

Contextual morphometrics are statistics of the primary morphometrics in a local spatial extent around each element of building and tesselation, hence, pure statistics to bring in more spatial variations, as opposed to new morphometrics.

In [ ]:
# Load saved morphometrics

primary = pd.read_parquet('primary.parquet')
geom = gpd.read_parquet('tessellation.pq', columns=["geometry"])

gdf = primary.set_index('uID')

spatial_weights = queen_3

unique_id = 'uID'

gdf = gdf.replace(np.inf, np.nan).fillna(0)  # normally does not happen, but to be sure
chars = gdf.columns


In [ ]:
# Check computed primary morphometrics by listing the names

chars


In [ ]:
# Define function for statistics

def theil(y):
    y = np.array(y)
    n = len(y)
    plus = y + np.finfo('float').tiny * (y == 0)  # can't have 0 values
    yt = plus.sum(axis=0)
    s = plus / (yt * 1.0)
    lns = np.log(n * s)
    slns = s * lns
    t = sum(slns)
    return t

def _simpson_di(data):

    def p(n, N):
        if n == 0:
            return 0
        return float(n) / N

    N = sum(data.values())

    return sum(p(n, N) ** 2 for n in data.values() if n != 0)


In [ ]:
# Call functions

skewness = pd.DataFrame(index=chars)
for c in chars:
    skewness.loc[c, 'skewness'] = sp.stats.skew(gdf[c])
headtail = list(skewness.loc[skewness.skewness >= 1].index)
to_invert = skewness.loc[skewness.skewness <= -1].index

for inv in to_invert:
    gdf[inv + '_r'] = gdf[inv].max() - gdf[inv]
inverted = [x for x in gdf.columns if '_r' in x]
headtail = headtail + inverted
natural = [x for x in chars if x not in headtail]


In [ ]:
# Create placeholders to save major stats

bins = {}
for c in headtail:
    bins[c] = mapclassify.HeadTailBreaks(gdf[c]).bins
for c in natural:
    bins[c] = mapclassify.gadf(gdf[c], method='NaturalBreaks')[1].bins

means = {}
ranges = {}
theils = {}
simpsons = {}

for ch in gdf.columns:
    means[ch] = []
    ranges[ch] = []
    theils[ch] = []
    simpsons[ch] = []


In [ ]:
# Save major stats

for index, _ in tqdm(gdf.iterrows(), total=gdf.shape[0]):
    #print(index)
    neighbours = [index]
    neighbours += spatial_weights.neighbors[index]

    subset = gdf.loc[neighbours]
    for ch in chars:
        values_list = subset[ch]
        idec = limit_range(values_list, rng=(10, 90))
        iquar = limit_range(values_list, rng=(25, 75))

        means[ch].append(np.mean(iquar))
        ranges[ch].append(max(iquar) - min(iquar))
        theils[ch].append(theil(idec))

        sample_bins = classifiers.UserDefined(values_list, list(bins[ch]))
        counts = dict(zip(bins[ch], sample_bins.counts))
        simpsons[ch].append(_simpson_di(counts))


In [ ]:
# Attach to the original dataframe table in one piece

contextual = {}
for ch in chars:
    print(ch)
    contextual[ch + '_meanIQ3'] = means[ch]
    contextual[ch + '_rangeIQ3'] = ranges[ch]
    contextual[ch + '_theilID3'] = theils[ch]
    contextual[ch + '_simpson'] = simpsons[ch]

contextual = pd.DataFrame(contextual, index=gdf.index)


In [ ]:
# Save weights

#import scipy
import pickle

#scipy.sparse.save_npz("w10.npz", spatial_weights.sparse)
with open('spatial_weights.pickle', 'wb') as handle:
    pickle.dump(spatial_weights, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# Save the main contextual results

contextual.to_parquet('contextual.parquet')
